In [ ]:
# Ucloud에서 실행했었던 코드

In [9]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -U scikit-learn
!pip install -U sentence-transformers scikit-learn imbalanced-learn
!pip install -U sentence-transformers

Looking in indexes: https://download.pytorch.org/whl/cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 76.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 99.1 MB/s  0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.1
    Uninstalling huggingface-hub-1.0.1:
      Successfully uninstalled huggingface-hub-1.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [sentence-transformers]ence-transformers]


In [10]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.5.1+cu121
True


In [14]:
# ClusterCentroids로 Train 데이터셋만 언더샘플링 (Sentence-BERT 임베딩 + KMeans 대표 샘플 추출)
# - test/val 은 손대지 않음 (현실 분포 유지)
# - 빈 클러스터/초과 할당/라운딩 잔여 분배 모두 안전 처리

import os
import random
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import normalize
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import pairwise_distances
from collections import Counter, defaultdict
from sentence_transformers import SentenceTransformer

# ====== CONFIG ======
SPLIT_DIR    = "Data"              # 이미 40~256 토큰 필터링된 split 파일 경로 (train.csv, val.csv, test.csv)
SAVE_DIR     = "Data/split_cc"     # 결과 저장 디렉터리
MODEL_NAME   = "sentence-transformers/all-MiniLM-L6-v2"  # 384-d sentence embedding
RANDOM_STATE = 42

# 임베딩/클러스터링 관련
EMB_BATCH       = 512      # GPU면 1024도 가능, CPU면 256~512 권장
KMEANS_STAGE1   = 3000     # 1단계 거친 KMeans 클러스터 개수 (클래스당). 2000~5000 권장

# 언더샘플링 타깃: 각 클래스당 보존 개수 상하한
MIN_PER_CLASS   = 10000    # 너무 작아지지 않도록 최소 보장
MAX_PER_CLASS   = 120000   # 너무 커지지 않도록 상한 (None이면 최소클래스 기준)

os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")
os.makedirs(SAVE_DIR, exist_ok=True)

# ====== 유틸/시드 ======
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
set_seed(RANDOM_STATE)

# ====== 1) Split 불러오기 ======
train_df = pd.read_csv(os.path.join(SPLIT_DIR, "train.csv"))
val_df   = pd.read_csv(os.path.join(SPLIT_DIR, "val.csv"))
test_df  = pd.read_csv(os.path.join(SPLIT_DIR, "test.csv"))

for d in (train_df, val_df, test_df):
    d["mbti"] = d["mbti"].astype(str).str.upper().str.strip()
    d["body"] = d["body"].astype(str)

print(f"loaded (filtered) -> train:{len(train_df):,}  val:{len(val_df):,}  test:{len(test_df):,}")

# ====== 2) Sentence-BERT 임베딩 (train만) ======
device = "cuda" if torch.cuda.is_available() else "cpu"
st_model = SentenceTransformer(MODEL_NAME, device=device)

train_texts = train_df["body"].tolist()
X = st_model.encode(
    train_texts,
    batch_size=EMB_BATCH,
    show_progress_bar=True,
    convert_to_numpy=True
)
# 코사인 거리 ≈ L2 정규화 후 유클리드 거리
X = normalize(X, norm="l2", axis=1).astype(np.float32)
y = train_df["mbti"].values

# ====== 3) 클래스별 타깃 수 결정 ======
counts = Counter(y)
min_count = min(counts.values())

if MAX_PER_CLASS is None:
    # 소수 클래스 크기 이상, MIN_PER_CLASS 이상으로
    target_per_class_value = max(MIN_PER_CLASS, min_count)
else:
    # 소수 클래스 크기를 [MIN_PER_CLASS, MAX_PER_CLASS] 범위로 클립
    target_per_class_value = int(np.clip(min_count, MIN_PER_CLASS, MAX_PER_CLASS))

# 각 클래스 타깃 = min(타깃값, 해당 클래스 실제 개수)  (소수 클래스는 그대로 보존)
target_per_class = {c: min(target_per_class_value, counts[c]) for c in counts}

# ====== 4) 클래스별 1단계 KMeans → 클러스터별로 대표 샘플 선택 ======
cls_to_idx = defaultdict(list)
for i, c in enumerate(y):
    cls_to_idx[c].append(i)

selected_idx = []

for c, idxs in cls_to_idx.items():
    idxs = np.array(idxs, dtype=np.int64)
    k_target = int(target_per_class[c])

    # 이미 작거나 소수 클래스면 전부 보존
    if len(idxs) <= k_target:
        selected_idx.extend(idxs.tolist())
        continue

    Xc = X[idxs]

    # 1단계: 거친 KMeans (클래스 내부를 크게 k1개로 묶음)
    k1 = max(1, min(KMEANS_STAGE1, len(idxs)))
    km = MiniBatchKMeans(
        n_clusters=k1,
        random_state=RANDOM_STATE,
        batch_size=4096,
        n_init=1,
        max_iter=50,
        init="k-means++"
    )
    labels = km.fit_predict(Xc)
    centers = km.cluster_centers_

    # 실제로 존재하는 (비어있지 않은) 클러스터만 사용
    present = np.unique(labels)
    centers = centers[present]

    cluster_sizes = np.array([(labels == cl).sum() for cl in present], dtype=np.int64)
    total_size = int(cluster_sizes.sum())

    # 이론상 발생하지 않지만 방어 코드: 전부 비어있으면 전역 중심에서 가까운 k_target개 선택
    if total_size == 0:
        gcenter = Xc.mean(axis=0, keepdims=True)
        d = pairwise_distances(Xc, gcenter, metric="euclidean").ravel()
        take_k = min(k_target, Xc.shape[0])
        pick_local = np.argsort(d)[:take_k]
        selected_idx.extend(idxs[pick_local].tolist())
        continue

    # (a) 클러스터 크기 비율로 k_target 배분 (실수)
    raw = cluster_sizes / total_size * k_target
    # (b) 내림한 정수로 우선 할당
    take = np.floor(raw).astype(int)
    # (c) 클러스터 보유량으로 cap
    take = np.minimum(take, cluster_sizes)

    # (d) 남은 몫을 큰 잔여부터 추가, cap 준수
    while take.sum() < k_target:
        remaining = k_target - int(take.sum())
        frac = raw - take
        order = np.argsort(-frac)  # 잔여가 큰 순
        added = 0
        for oi in order:
            if take[oi] < cluster_sizes[oi]:
                take[oi] += 1
                added += 1
                if added == remaining:
                    break
        if added == 0:
            break  # 더 못 채우면 종료

    # (e) 각 (비어있지 않은) 클러스터별로 center에 가장 가까운 실제 샘플 need개 선택
    for local_i, cl in enumerate(present):
        need = int(take[local_i])
        if need <= 0:
            continue
        cl_mask = (labels == cl)
        if not np.any(cl_mask):
            continue
        cl_idxs = idxs[cl_mask]
        Xcl = Xc[cl_mask]
        # cap by available
        need = min(need, Xcl.shape[0])

        d = pairwise_distances(Xcl, centers[local_i][None, :], metric="euclidean").ravel()
        pick_local = np.argsort(d)[:need]
        selected_idx.extend(cl_idxs[pick_local].tolist())

# 최종 고유 인덱스 정렬
selected_idx = sorted(set(selected_idx))
train_balanced_df = train_df.iloc[selected_idx].copy().reset_index(drop=True)

# ====== 5) 저장 ======
train_balanced_df.to_csv(os.path.join(SAVE_DIR, "train_cc.csv"), index=False)
val_df.to_csv(os.path.join(SAVE_DIR, "val.csv"), index=False)
test_df.to_csv(os.path.join(SAVE_DIR, "test.csv"), index=False)

# 선택된 인덱스(원본 train 내 row 인덱스)도 보관하면 재현성에 도움
np.save(os.path.join(SAVE_DIR, "train_cc_selected_idx.npy"), np.array(selected_idx, dtype=np.int64))

# ====== 6) 로그 ======
print("\n[train original counts]")
print(train_df["mbti"].value_counts().sort_values(ascending=False))

print("\n[train after CC counts]")
print(train_balanced_df["mbti"].value_counts().sort_values(ascending=False))

print(f"\nSizes -> train_cc: {len(train_balanced_df):,} | val: {len(val_df):,} | test: {len(test_df):,}")
print(f"Selected ratio: {len(train_balanced_df):,} / {len(train_df):,} "
      f"= {len(train_balanced_df)/max(1,len(train_df)):.3f}")

loaded (filtered) -> train:2,422,635  val:807,545  test:807,546


Batches:   0%|          | 0/4732 [00:00<?, ?it/s]


[train original counts]
mbti
INFP    523188
INTP    490228
INFJ    425456
INTJ    331113
ENTP    219392
ENTJ    121832
ISTP     77579
ENFP     77093
ENFJ     54852
ISFJ     28537
ISTJ     22268
ISFP     16829
ESTP     15859
ESFJ      7196
ESTJ      7107
ESFP      4106
Name: count, dtype: int64

[train after CC counts]
mbti
INTJ    10000
ENTP    10000
INFJ    10000
ISFJ    10000
ENFJ    10000
ISTJ    10000
INFP    10000
ENTJ    10000
ENFP    10000
ISFP    10000
INTP    10000
ISTP    10000
ESTP    10000
ESFJ     7196
ESTJ     7107
ESFP     4106
Name: count, dtype: int64

Sizes -> train_cc: 148,409 | val: 807,545 | test: 807,546
Selected ratio: 148,409 / 2,422,635 = 0.061
